In [ ]:
# Setup and imports
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
from IPython import display
import seaborn as sns
sns.set_theme(style="whitegrid")

import numpy as np
from scipy.stats import norm, \
    beta, cauchy, expon, rayleigh, uniform
import pandas as pd

# Lesson 6: Introduction to Copulas

This notebook introduces copulas, the mathematical objects that allow us to model dependence structures separately from marginal distributions. We will cover Sklar's Theorem, fundamental properties of copulas, and demonstrate how different copulas produce different dependence patterns.

## Learning Objectives
- Understand Sklar's Theorem and its implications
- Learn the fundamental properties of copulas
- Visualize how copulas separate dependence from marginal behavior

## Sklar's Theorem

Sklar's Theorem is the foundational result in copula theory. It states that any multivariate joint distribution can be written in terms of its marginal distributions and a copula function that captures the dependence structure.

**Theorem (Sklar, 1959):** Let $H$ be a joint distribution function with marginals $F$ and $G$. Then there exists a copula $C$ such that:

$$ H(x,y) = C(F(x), G(y)) $$

Conversely, if we know the copula and the marginals, we can reconstruct the joint distribution:

$$ x = F^{-1}(u), \quad y = G^{-1}(v) $$
$$ C(u,v) = H(F^{-1}(u), G^{-1}(v)) $$

This theorem tells us that we can study dependence (via copulas) separately from marginal behavior.

## Properties of Copulas

A copula $C: [0,1]^2 \rightarrow [0,1]$ must satisfy the following properties:

**Boundary Conditions:**
$$ C(u,0) = C(0,v) = 0 $$
$$ C(u,1) = u $$
$$ C(1,v) = v $$

**2-Increasing Property:** For any $u_1 \leq u_2$ and $v_1 \leq v_2$:
$$ C(u_2,v_2) - C(u_2,v_1) - C(u_1,v_2) + C(u_1,v_1) \geq 0 $$

This last property ensures that the probability of any rectangle in the unit square is non-negative.

## Frechet-Hoeffding Bounds

Every copula is bounded by the Frechet-Hoeffding bounds. These represent the extreme cases of perfect positive and negative dependence.

**Upper Bound (Perfect Positive Dependence / Comonotonicity):**
$$ M(u,v) = \min(u,v) $$

**Lower Bound (Perfect Negative Dependence / Countermonotonicity):**
$$ W(u,v) = \max(u+v-1, 0) $$

For any copula $C$:
$$ W(u,v) \leq C(u,v) \leq M(u,v) $$

## Independence Copula

When variables are independent, the copula takes a particularly simple form:
$$ \Pi(u,v) = u \cdot v $$

This corresponds to the case where the joint distribution is simply the product of the marginals.

In [ ]:
# Visualize comonotonic and countermonotonic relationships
# Comonotonic: variables increase together (perfect positive dependence)
# Countermonotonic: one increases while the other decreases (perfect negative dependence)

plt.figure()
x = np.linspace(1, 10, 200)

# Countermonotonic examples: decreasing relationships
plt.subplot(1, 2, 1)
plt.plot(x, np.exp(-x), label='$e^{-x}$')
plt.plot(x, -0.1*x, label='$-0.1x$')
plt.plot(x, -x**(0.2), label='$-x^{0.2}$')
plt.title('Countermonotonic Relationships')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()

# Comonotonic examples: increasing relationships
plt.subplot(1, 2, 2)
plt.plot(x, np.exp(0.2*x), label='$e^{0.2x}$')
plt.plot(x, x, label='$x$')
plt.plot(x, np.log(x), label='$\ln(x)$')
plt.title('Comonotonic Relationships')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()

plt.tight_layout()

## Copula Density

The density of a joint distribution can be decomposed into the copula density and the marginal densities:

$$ h(x,y) = c(F(x),G(y)) \cdot f(x) \cdot g(y) $$

where:
- $h(x,y)$ is the joint density
- $c(u,v) = \frac{\partial^2 C(u,v)}{\partial u \partial v}$ is the copula density
- $f(x)$ and $g(y)$ are the marginal densities

This decomposition shows how copulas separate the dependence structure from the marginal behavior.

In [ ]:
# Generate samples from two different copulas to compare their dependence structures
# This demonstrates that different copulas produce different joint behaviors

# Gaussian Copula: symmetric dependence, no tail dependence
P = np.asarray([
    [1, 0.99],      # Correlation matrix with rho = 0.99
    [0.99, 1]
])
d = P.shape[0]      # Dimension
n = 500             # Number of samples

# Generate Gaussian copula samples using Cholesky decomposition
A = np.linalg.cholesky(P)
Z = np.random.normal(size=(n, d))
U_Gauss = norm.cdf(np.matmul(Z, A))

# Clayton Copula: asymmetric dependence, lower tail dependence
alpha = 6           # Clayton parameter (higher = stronger dependence)

# Generate Clayton copula samples using conditional distribution method
u = np.random.rand(n)
t = np.random.rand(n)
v = ((t / u**(-alpha-1))**(-alpha/(1+alpha)) - u**(-alpha) + 1)**(-1/alpha)
U_Clayton = np.asarray([u, v]).T

In [ ]:
# Transform copula samples to standard normal marginals
# This allows us to compare the dependence structures visually

H1 = np.empty_like(U_Gauss)
H2 = np.empty_like(U_Clayton)

# Apply inverse normal CDF (quantile function) to get Normal marginals
H1[:, 0] = norm.ppf(U_Gauss[:, 0])
H1[:, 1] = norm.ppf(U_Gauss[:, 1])
H2[:, 0] = norm.ppf(U_Clayton[:, 0])
H2[:, 1] = norm.ppf(U_Clayton[:, 1])

# Plot comparison
plt.figure()

plt.subplot(1, 2, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.5)
plt.xlabel('$\mathcal{N}(0,1)$')
plt.ylabel('$\mathcal{N}(0,1)$')
plt.title('Gaussian Copula $\\rho=%0.02f$' % (P[0, 1],))

plt.subplot(1, 2, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.5)
plt.xlabel('$\mathcal{N}(0,1)$')
plt.ylabel('$\mathcal{N}(0,1)$')
plt.title('Clayton Copula $\\alpha=%0.02f$' % (alpha,))

plt.tight_layout()

In [ ]:
# Demonstrate how the same copulas produce different joint distributions
# when combined with different marginals (Beta and Cauchy)

H1 = np.empty_like(U_Gauss)
H2 = np.empty_like(U_Clayton)

# Transform to Beta(2,5) and Cauchy(0,1) marginals
H1[:, 0] = beta.ppf(U_Gauss[:, 0], 2, 5)
H1[:, 1] = cauchy.ppf(U_Gauss[:, 1])
H2[:, 0] = beta.ppf(U_Clayton[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_Clayton[:, 1])

# Plot comparison with different marginals
plt.figure()

plt.subplot(1, 2, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.5)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')
plt.title('Gaussian Copula $\\rho=%0.02f$' % (P[0, 1],))

plt.subplot(1, 2, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.5)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')
plt.title('Clayton Copula $\\alpha=%0.02f$' % (alpha,))

plt.tight_layout()